In [38]:
import numpy as np
import pandas as pd
import cantera as ct
import os


cti_file='multifacet_original.yaml'
new_file='mod.yaml'
file_copy="".join(('cp ',cti_file,' ', new_file))
os.popen(file_copy)

input_file = open(new_file,'r')
lines = input_file.readlines()
input_file.close()
new_file_content=""

switch=False
for index,line in enumerate(lines):
    if line.strip().startswith('- equation: O(111) + site(211) <=> site(111) + O(211)'):
        switch=True
        line_idx=index
    elif index == line_idx+1:
        if switch==True:
            initial=line.strip()
            old=initial.split('Ea:')
            newEa=float(old[1].strip().replace("}",""))
            newEa+=10
            modified = old[0] + 'Ea: ' + str(newEa) + '}'
            new_line=line.replace(initial,modified)
            new_file_content += new_line
            print(new_line)
        else:
            initial=line.strip()
            new_line=line.replace(initial,initial)
            new_file_content += new_line
    else:
        switch=False
        initial=line.strip()
        new_line=line.replace(initial,initial)
        new_file_content += new_line

writing_file = open(new_file, "w")
writing_file.write(new_file_content)
writing_file.close()
    

  rate-constant: {A: 9.5e16, b: 0, Ea: 55.7}



In [7]:
import shutil
import pandas as pd

cdiff111_bee=pd.read_csv('/Users/bjkreitz/work/multifacet/TPD/DFT/ensembles/Ni111/C_diff_bee.txt', sep="\t", header=0)
codiff111_bee=pd.read_csv('/Users/bjkreitz/work/multifacet/TPD/DFT/ensembles/Ni111/CO_diff_bee.txt', sep="\t", header=0)    
odiff111_bee=pd.read_csv('/Users/bjkreitz/work/multifacet/TPD/DFT/ensembles/Ni111/O_diff_bee.txt', sep="\t", header=0)   

cdiff100_bee=pd.read_csv('/Users/bjkreitz/work/multifacet/TPD/DFT/ensembles/Ni100/C_diff_bee.txt', sep="\t", header=0)
codiff100_bee=pd.read_csv('/Users/bjkreitz/work/multifacet/TPD/DFT/ensembles/Ni100/CO_diff_bee.txt', sep="\t", header=0)    
odiff100_bee=pd.read_csv('/Users/bjkreitz/work/multifacet/TPD/DFT/ensembles/Ni100/O_diff_bee.txt', sep="\t", header=0)   


# Function to modify lines based on specific patterns
def modify_lines(lines, patterns, delta_Ea):
    new_file_content = ""
    switch = False

    for index, line in enumerate(lines):
        if any(line.strip().startswith(pattern) for pattern in patterns):
            switch = True
            new_file_content += line
            line_idx = index
        elif switch and index == line_idx + 1:
            initial = line.strip()
            old = initial.split('Ea:')
            newEa = float(old[1].strip().replace("}", ""))
            newEa += delta_Ea
            newEa *= 1e6
            modified = old[0] + 'Ea: ' + str(newEa) + '}'
            new_line = line.replace(initial, modified)
            new_file_content += new_line
            switch = False  # Reset switch after modification
        else:
            new_file_content += line

    return new_file_content

# Define the patterns and corresponding Ea adjustments
patterns_list = [
    (['- equation: O(111) + site(211) <=> site(111) + O(211)', '- equation: O(111) + site(110) <=> site(111) + O(110)'], odiff111_bee['delta_Ea'][0]),
    (['- equation: OC(111) + site(211) <=> site(111) + OC(211)', '- equation: OC(111) + site(110) <=> site(111) + OC(110)'], codiff111_bee['delta_Ea'][0]),
    (['- equation: C(111) + site(211) <=> site(111) + C(211)', '- equation: C(111) + site(110) <=> site(111) + C(110)'], cdiff111_bee['delta_Ea'][0]),
    (['- equation: O(100) + site(211) <=> site(100) + O(211)', '- equation: O(100) + site(110) <=> site(100) + O(110)'], odiff100_bee['delta_Ea'][0]),
    (['- equation: OC(100) + site(211) <=> site(100) + OC(211)', '- equation: OC(100) + site(110) <=> site(100) + OC(110)'], codiff100_bee['delta_Ea'][0]),
    (['- equation: C(100) + site(211) <=> site(100) + C(211)', '- equation: C(100) + site(110) <=> site(100) + C(110)'], cdiff100_bee['delta_Ea'][0]),
]

cti_file = 'mech_uq.yaml'
new_filename = 'mech.yaml'

shutil.copyfile(cti_file, new_file)

num_pts = 10 # the number of points to generate 
start=0

directory='mechanisms/File_'

for k in range(num_pts):
    # Copy the file
    #Create a new folder for the new input file and write the input file
    mkdir_writer="".join((directory,str(k+start)))
    new_file=mkdir_writer + '/' + new_filename
    
    shutil.copyfile(cti_file, new_file)
    
    # Open the file and read the lines
    with open(new_file, 'r') as input_file:
        lines = input_file.readlines()

    # Apply modifications for each pattern set
    for patterns, delta_Ea in patterns_list:
        lines = modify_lines(lines, patterns, delta_Ea).splitlines(keepends=True)

    # Write the final modified content back to the file
    with open(new_file, "w") as writing_file:
        writing_file.write("".join(lines))